In [ ]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import googletrans
from googletrans import Translator
import re
import sys
import warnings

In [ ]:
file_path = "D:\\machine_learning_data\\train.csv"
raw_data_train = pd.read_csv(file_path)
file_path = "D:\\machine_learning_data\\test.csv"
raw_data_test = pd.read_csv(file_path)
real_test_data = pd.read_csv("D:\\machine_learning_data\\data_real.csv")

In [ ]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

#real_test_data = real_test_data.drop(real_test_data.index[0])
real_test_data = real_test_data.drop(real_test_data.index[288])
real_test_data = real_test_data.drop(real_test_data.index[287])
real_test_data = real_test_data.drop(real_test_data.index[270])
translator = Translator()

count = 0
for element in real_test_data['post_description']:
    print(count)
    real_test_data.iloc[count, 0] = translator.translate(deEmojify(element)).text
    count = count + 1

In [ ]:
data_train = raw_data_train
data_test = raw_data_test

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def split(string, start, end):
    return ' '.join(string.split()[start:end])

# cleaning training dataset

data_train['post'] = data_train['post'].str.lower()
#data_train['post'] = data_train['post'].apply(cleanHtml)
data_train['post'] = data_train['post'].apply(cleanPunc)
data_train['post'] = data_train['post'].apply(keepAlpha)

count = 0
for str in data_train['post']:
    data_train.iloc[count, 0] = split(str, 0, 50)
    count = count + 1
    
# cleaning test dataset

data_test['post'] = data_test['post'].str.lower()
#data_test['post'] = data_test['post'].apply(cleanHtml)
data_test['post'] = data_test['post'].apply(cleanPunc)
data_test['post'] = data_test['post'].apply(keepAlpha)

count = 0
for str in data_test['post']:
    data_test.iloc[count, 0] = split(str, 0, 50)
    count = count + 1


In [ ]:
nltk.download("stopwords")

In [ ]:
stop_words = set(stopwords.words('english'))
#stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten',
#                   'may','also','across','among','beside','however','yet','within'])
#re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
new_stop_words = ['zero','one','two','three','four','five','six','seven','eight','nine','ten',
                  'may','also','across','among','beside','however','yet','within', 'r', 'function', 'code', 
                  'library', 'libraries']
stop_words = stop_words.union(new_stop_words)

not_stopwords = {'which', 'when', 'how', 'but', 'should', 'where', 'who', 'whom', 'what'}
stop_words = set([word for word in stop_words if word not in not_stopwords])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data_train["post"] = data_train["post"].apply(removeStopWords)
data_test["post"] = data_test["post"].apply(removeStopWords)

In [ ]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data_train['post'] = data_train['post'].apply(stemming)
data_test['post'] = data_test['post'].apply(stemming)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

train_text = data_train['post']
test_text = data_test['post']

vectorizer.fit_transform(train_text)
vectorizer.fit_transform(test_text)

x_train = vectorizer.transform(train_text)
y_train = data_train.drop(labels = ['post'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = data_test.drop(labels = ['post'], axis=1)


In [ ]:
# MultiClass Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

categories = list(raw_data_train.columns.values)
categories = categories[1:]
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, data_train[category])
    
    # calculating test accuracy
    prediction_test = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(data_test[category], prediction_test)))
    

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

In [ ]:
# using MLkNN
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()
# train
classifier_new.fit(x_train, y_train)
# predict
predictions_new = classifier_new.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))

In [ ]:
# calculating the predictions for the real dataset
data = real_test_data

data['post_description'] = data['post_description'].str.lower()
# data['post_description'] = data['post_description'].apply(cleanHtml)
data['post_description'] = data['post_description'].apply(cleanPunc)
data['post_description'] = data['post_description'].apply(keepAlpha)
data['post_description'] = data['post_description'].apply(removeStopWords)
data['post_description'] = data['post_description'].apply(stemming)
# print(data)
count = 0

for str in data['post_description']:
    data.iloc[count, 0] = split(str, 0, 50)
    count = count + 1

real_text = data['post_description']
print(real_text)
vectorizer.fit_transform(real_text)
print(real_text)
test_data = vectorizer.transform(real_text)
prediction_real_data = LogReg_pipeline.predict(test_data)

In [ ]:
print(test_data)
print(x_test)